# Sprint 6: Deployment Simulation & Conclusions

**Objective:** Simulate the production inference pipeline using real clinical patient data.
**Key Changes:**
- Loads the optimized `best_pipeline`.
- Uses a **Real Patient Profile** (Biomarkers) instead of survey responses.
- Predicts physiological risk class and probability.

In [ ]:
import pandas as pd
import numpy as np
from pycaret.classification import load_model, predict_model
import os

# 1. Load Model
model_path = "../models/best_pipeline"
if not os.path.exists(model_path + ".pkl"):
    model_path = "models/best_pipeline" # Fallback

try:
    pipeline = load_model(model_path)
    print("✅ Pipeline loaded successfully.")
except Exception as e:
    print(f"❌ Failed to load pipeline: {e}")
    pipeline = None

## 2. Simulate Real Patient Input

In [ ]:
# Real Patient Data (Numeric Values - NO STRINGS)
patient_data = {
    'Sex': 1,               # Male
    'SystolicBP': 140.5,    # High Blood Pressure
    'TotalCholesterol': 220.0,# Borderline High
    'LDL': 130.0,           # High
    'Triglycerides': 150.0, # Normal/Borderline
    'HbA1c': 5.7,           # Prediabetic
    'Glucose': 100.0,       # Normal
    'UricAcid': 6.0,        # Normal
    'Creatinine': 1.0,      # Normal
    'BMI': 28.5,            # Overweight
    'WaistCircumference': 95.0,
    'Height': 175.0,        # cm
    'Smoking': 0,           # Non-smoker
    'Alcohol': 1,           # Drinker
    'PhysicalActivity': 0,  # Sedentary
    'HealthInsurance': 1    # Insured
}

# Convert to DataFrame
input_df = pd.DataFrame([patient_data])

print("Patient Profile:")
display(input_df)

## 3. Generate Prediction

In [ ]:
if pipeline:
    try:
        # Predict
        pred = predict_model(pipeline, data=input_df)
        
        print("\n--- Inference Result ---")
        # Extract prediction columns 
        cols = pred.columns
        label_col = [c for c in cols if 'label' in c.lower() or 'prediction' in c.lower()][0]
        score_col = [c for c in cols if 'score' in c.lower()][0]
        
        label = pred[label_col].iloc[0]
        score = pred[score_col].iloc[0]
        
        print(f"Risk Class: {label} (1=High Risk, 0=Low Risk)")
        print(f"Probability: {score:.4f}")
        
        if label == 1:
            print("🚨 ALERT: High physiological risk detected.")
        else:
            print("✅ LOW RISK: Biomarkers within safe parameters.")
            
    except Exception as e:
        print(f"Inference Error: {e}")